In [1]:
import os
import cv2
import shutil
import pandas as pd
import numpy as np

from colorama import Fore, Style
from typing import List
from dotenv import load_dotenv
from DataRepository import DataRepository
from ultralytics import YOLO

load_dotenv()
STORAGE_DIR = os.getenv("STORAGE_DIR")
CROPPED_VIDEOS_FOLDER = "cropped-videos"
# IMAGES_FOLDER = "images"
# LABELS_FOLDER = "labels"
SUPPORTED_VIDEO_FORMATS = os.getenv("SUPPORTED_VIDEO_FORMATS")
SUPPORTED_IMAGE_FORMATS = os.getenv("SUPPORTED_IMAGE_FORMATS")
VIDEO_IMAGE_PREVIEW_FOLDER = os.getenv("VIDEO_IMAGE_PREVIEW_FOLDER")


In [2]:
print(STORAGE_DIR), print(CROPPED_VIDEOS_FOLDER)
# computervision/labeled_frame_nrs_to_imgs.ipynb

/media/miked/Elements/Judge/FINISHED-DB-READY
cropped-videos


(None, None)

In [3]:
os.makedirs(os.path.join(STORAGE_DIR, CROPPED_VIDEOS_FOLDER), exist_ok=True)

def get_video_path(repo, videoId):
    return os.path.join(STORAGE_DIR, repo.VideoNames.loc[videoId, "name"])

In [37]:
repo = DataRepository()
predictions_per_second = 2000
videoId = 1419
DIM = 720
videoPath = get_video_path(repo, videoId=videoId)
videoOutputPath = os.path.join(STORAGE_DIR, CROPPED_VIDEOS_FOLDER, f"{DIM}_{videoId}.mp4")
videoPath, videoOutputPath

connection established, loading frame paths
rel paths of framelabels loaded


('/media/miked/Elements/Judge/FINISHED-DB-READY/competition/belgium/DD3/2022-dd3-belgium-grand-nationals-20.mp4',
 '/media/miked/Elements/Judge/FINISHED-DB-READY/cropped-videos/720_1419.mp4')

In [38]:
modeltrainround = 2
modelpath = f"/home/miked/code/judge/runs/detect/train{modeltrainround}"
modelpath

'/home/miked/code/judge/runs/detect/train2'

In [39]:
# From https://docs.ultralytics.com/modes/predict/#key-features-of-predict-mode
# Load a model
model = YOLO(os.path.join(modelpath, "weights", "best.pt"))


In [ ]:
cap = cv2.VideoCapture(videoPath)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
wait_time = int(333 / fps - 7)
frames_to_skip = fps // predictions_per_second
frames_to_skip
padding_x = width // 80
padding_y = height // 65

smootval = 0.85
smooted_x1_min = None
smooted_y1_min = None
smooted_x2_max = None
smooted_y2_max = None

max_w = 0
max_h = 0
min_w = width
min_h = height
max_wh = max(width, height)

i = 0
times_with_no_jumper = 0

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
out = cv2.VideoWriter(videoOutputPath, fourcc, fps, (DIM, DIM))
ret, frame = cap.read()
while ret:
    i += 1
    result = model(frame, verbose=False)
    xyxy_boxes = result[0].boxes.xyxy
    if xyxy_boxes.shape[0] > 0:
        x1_min = max(0, int(xyxy_boxes[:, 0].min().item()) - padding_x)
        y1_min = max(0, int(xyxy_boxes[:, 1].min().item()) - padding_y)
        x2_max = min(width, int(xyxy_boxes[:, 2].max().item()) + padding_x)
        y2_max = min(height, int(xyxy_boxes[:, 3].max().item()) + padding_y)
        times_with_no_jumper = 0
    elif times_with_no_jumper < 8:
        times_with_no_jumper += 1
    else:
        x1_min = 0
        y1_min = 0
        x2_max = width
        y2_max = height
        times_with_no_jumper += 1

    if smooted_x1_min is None:
        smooted_x1_min = x1_min
        smooted_y1_min = y1_min
        smooted_x2_max = x2_max
        smooted_y2_max = y2_max
    else:
        smooted_x1_min = int(smootval * smooted_x1_min + (1-smootval) * x1_min)
        smooted_y1_min = int(smootval * smooted_y1_min + (1-smootval) * y1_min)
        smooted_x2_max = int(smootval * smooted_x2_max + (1-smootval) * x2_max)
        smooted_y2_max = int(smootval * smooted_y2_max + (1-smootval) * y2_max)

    w_jumpers = smooted_x2_max - smooted_x1_min
    h_jumpers = smooted_y2_max - smooted_y1_min
    max_w = max(max_w, w_jumpers)
    max_h = max(max_h, h_jumpers)
    min_w = min(min_w, w_jumpers)
    min_h = min(max_h, h_jumpers)

    cropped_frame = frame[smooted_y1_min:smooted_y2_max, smooted_x1_min:smooted_x2_max]
    max_wh_jumpers = max(w_jumpers, h_jumpers)
    zeros = np.zeros((max_wh_jumpers, max_wh_jumpers, 3), dtype=np.uint8)
    offset_x = (max_wh_jumpers - w_jumpers) // 2
    offset_y = (max_wh_jumpers - h_jumpers) // 2
    zeros[offset_y:offset_y+h_jumpers, offset_x:offset_x+w_jumpers] = cropped_frame
    zeros = cv2.resize(zeros, (DIM, DIM))

    out.write(zeros)
    
    # cv2.imshow('Video', zeros)

    # # Skip frames
    # s = frames_to_skip
    # while ret and s > 0:
    #     ret, _ = cap.read()
    #     s -= 1
    
    # if cv2.waitKey(wait_time) & 0xFF == ord('q'):
    #     break
    ret, frame = cap.read()

print("min width", min_w, "max_w", max_w)
print("min_height", min_h, "max_h", max_h)
print(i)

# Release everything if job is finished
out.release()
cap.release()
cv2.destroyAllWindows()